In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import pandas as pd
import datetime as dt

In [3]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [4]:
??proc_df

In [5]:
path = "data/sapflow/"

In [7]:
weather_csv = pd.read_csv(f'{path}all_weather.txt', 
                          low_memory=False, parse_dates=[["Date","Time"]], encoding='ISO-8859-1')

sapflow_csv = pd.read_csv(f'{path}all_sapflow.txt', 
                          low_memory=False, parse_dates=[["Date","Time"]], encoding='ISO-8859-1')

In [8]:
weather_csv["Date_Time"] = weather_csv["Date_Time"].dt.strftime('%d/%m/%Y %H:%M')
sapflow_csv["Date_Time"] = sapflow_csv["Date_Time"].dt.strftime('%d/%m/%Y %H:%M')

In [9]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [ ]:
display_all(weather_csv)

In [10]:
weather_csv = weather_csv[["Date_Time", "Solar Radiation (W/m^2)", "Air Temperature (deg.C)", "Corrected RH (%)", "Wind Speed (m/s)"]]
weather_csv.rename(columns = {"Date_Time":"datetime", "Solar Radiation (W/m^2)":"solar_rad","Air Temperature (deg.C)":"air_temp","Corrected RH (%)":"RH","Wind Speed (m/s)":"wind_speed"}, inplace=True)

In [11]:
sapflow_csv = sapflow_csv[["Date_Time", "Uncorrected Out (cm/hr)", "Uncorrected In (cm/hr)"]]
sapflow_csv.rename(columns ={"Date_Time":"datetime", "Uncorrected Out (cm/hr)":"outer_sapflow", "Uncorrected In (cm/hr)":"inner_sapflow"}, inplace=True)

In [12]:
merged=pd.merge(weather_csv, sapflow_csv, on='datetime', how='inner')

merged=merged.drop_duplicates()

In [ ]:
display_all(merged)

In [13]:
def label_time (row):
    hour = dt.datetime.strptime(row['datetime'],'%d/%m/%Y %H:%M').hour
    if hour >= 6 and hour < 12:
        return 'morning'
    if hour >= 12 and hour < 17:
        return 'afternoon'
    if hour >= 17 and hour < 20:
        return 'evening'
    if hour >= 20 and hour <= 24:
        return 'night'
    if hour >= 0 and hour < 6:
        return 'night'

In [ ]:
#merged.apply (label_time, axis=1)

In [14]:
merged['time_of_day'] = merged.apply (label_time, axis=1)

In [ ]:
display_all(merged)

In [15]:
df_outer = merged.drop("inner_sapflow", axis=1)
df_inner = merged.drop("outer_sapflow", axis=1)

In [16]:
df_outer = df_outer[pd.to_numeric(df_outer['outer_sapflow'], errors='coerce').notnull()]
df_inner = df_inner[pd.to_numeric(df_inner['inner_sapflow'], errors='coerce').notnull()]

In [17]:
df_outer['outer_sapflow'] = pd.to_numeric(df_outer['outer_sapflow'])
df_inner['inner_sapflow'] = pd.to_numeric(df_inner['inner_sapflow'])

In [18]:
outer_min = df_outer['outer_sapflow'].min()
inner_min = df_inner['inner_sapflow'].min()

In [19]:
df_outer['new_sapflow'] = df_outer.apply(lambda row: row['outer_sapflow'] + abs(outer_min), axis=1)
df_inner['new_sapflow'] = df_inner.apply(lambda row: row['inner_sapflow'] + abs(inner_min), axis=1)

Use to save csv files

In [20]:
merged.to_csv(f'{path}merged.csv', index =False)
df_outer.to_csv(f'{path}df_outer.csv', index =False)
df_inner.to_csv(f'{path}df_inner.csv', index =False)

 Not complete..... df_outer

In [21]:
df_trn, y_trn, nas = proc_df(df_outer, 'new_sapflow')

In [23]:
def split_vals(a,n): return a[:n], a[n:]
n_valid = 8000
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
raw_train, raw_valid = split_vals(df_outer, n_trn)

In [24]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [25]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.06501211279386168, 0.9299075229830537, 0.9999172940095571, 0.9851885255108048, 0.9997223482542263]
